<a href="https://colab.research.google.com/github/saipranathi2022-dot/Qiskit_multi_qubit_gates/blob/main/Qiskit_multi_qubit_gates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit qiskit-aer --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 140.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Initialize simulator
sim = AerSimulator()

# ====================================
# Original Circuit (for comparison)
# ====================================
print("\n--- Original Circuit ---")

qc_original = QuantumCircuit(2, 2)
qc_original.h(0)
qc_original.cx(0, 1)
qc_original.cz(0, 1)
qc_original.swap(0, 1)
qc_original.measure_all()

compiled_orig = transpile(qc_original, sim)
result_orig = sim.run(compiled_orig, shots=1024).result()
counts_orig = result_orig.get_counts()

print(qc_original.draw())
print("Measurement Counts (Original):", counts_orig)


# ====================================
# 1. Control / Target Variation
# ====================================
print("\nTask 1")

qc_ct = QuantumCircuit(2, 2)
qc_ct.h(0)
qc_ct.cx(1, 0)  # Control = qubit 1, Target = qubit 0
qc_ct.cz(1, 0)  # Control = 1, Target = 0
qc_ct.swap(0, 1)
qc_ct.measure_all()

compiled_ct = transpile(qc_ct, sim)
result_ct = sim.run(compiled_ct, shots=1024).result()
counts_ct = result_ct.get_counts()

print(qc_ct.draw())
print("Measurement Counts (Control/Target Swapped):", counts_ct)

print("\nExplanation: Changing control and target qubits alters how entanglement is created. The resulting output distribution changes because different qubits influence each other in different directions.")


# ====================================
# 2. Gate Removal (Remove SWAP)
# ====================================
print("\nTask 2")

qc_removed = QuantumCircuit(2, 2)
qc_removed.h(0)
qc_removed.cx(0, 1)
qc_removed.cz(0, 1)
# qc_removed.swap(0, 1)  # Removed
qc_removed.measure_all()

compiled_removed = transpile(qc_removed, sim)
result_removed = sim.run(compiled_removed, shots=1024).result()
counts_removed = result_removed.get_counts()

print(qc_removed.draw())
print("Measurement Counts (Without SWAP):", counts_removed)

print("\nExplanation: Removing the SWAP gate keeps the qubits in their original positions, so measurement results map to different classical bits compared to the swapped version.")


# ====================================
# 3. Add Another Hadamard (on qubit 1 before CNOT)
# ====================================
print("\nTask 3")

qc_h_added = QuantumCircuit(2, 2)
qc_h_added.h(0)
qc_h_added.h(1)      # New Hadamard on qubit 1
qc_h_added.cx(0, 1)
qc_h_added.cz(0, 1)
qc_h_added.swap(0, 1)
qc_h_added.measure_all()

compiled_h_added = transpile(qc_h_added, sim)
result_h_added = sim.run(compiled_h_added, shots=1024).result()
counts_h_added = result_h_added.get_counts()

print(qc_h_added.draw())
print("Measurement Counts (Hadamard Added on Qubit 1):", counts_h_added)

print("\nExplanation: Applying a Hadamard on the second qubit before entangling operations changes its basis. This affects how the two qubits interfere and entangle, leading to different measurement distributions.")



--- Original Circuit ---
        ┌───┐            ░ ┌─┐   
   q_0: ┤ H ├──■───■──X──░─┤M├───
        └───┘┌─┴─┐ │  │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──X──░──╫─┤M├
             └───┘       ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts (Original): {'00 00': 506, '11 00': 518}

Task 1
        ┌───┐┌───┐       ░ ┌─┐   
   q_0: ┤ H ├┤ X ├─■──X──░─┤M├───
        └───┘└─┬─┘ │  │  ░ └╥┘┌─┐
   q_1: ───────■───■──X──░──╫─┤M├
                         ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts (Control/Target Swapped): {'00 00': 514, '10 00': 510}

Explanation: Changing control and target qubits alters how entanglement is created. The resulting output distribution changes because different qubits influence each other in different directions.

Task 2
        ┌───┐ 